# 🇰🇷→🇯🇵 MarianMT 学習ノートブック

韓国語→日本語翻訳モデルの学習パイプライン

## 手順
1. 環境セットアップ
2. 教師データ生成 (NLLB-200)
3. MarianMT学習
4. ONNX変換・エクスポート

In [ ]:
import os

## 0. GPU確認

In [ ]:
!nvidia-smi

## 1. 環境セットアップ

In [ ]:
# RunPodの永続ディレクトリを設定
base_path = "/workspace/grasp-models"
os.makedirs(base_path, exist_ok=True)
%cd {base_path}

In [ ]:
!mkdir -p /workspace/huggingface_cache
!ls /workspace/huggingface_cache

In [ ]:
# キャッシュ先を容量の大きい /workspace 配下に変更
os.environ['HF_HOME'] = '/workspace/huggingface_cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = '/workspace/huggingface_cache'

In [ ]:
rm -rf /root/.cache/huggingface/*

In [ ]:
!df -h

In [ ]:
!pwd

In [ ]:
# リポジトリのクローン
# すでにフォルダが存在する場合は pull して最新にします
repo_url = "https://github.com/nakaikento/grasp-models.git"
repo_dir = "grasp-models"

if not os.path.exists(repo_dir):
    print(f"Cloning repository from {repo_url}...")
    !git clone {repo_url}
    %cd {repo_dir}
else:
    print("Repository already exists. Pulling latest changes...")
    %cd {repo_dir}
    !git pull

In [ ]:
# 依存関係インストール
!pip install -r requirements.txt

## 2. データ確認

In [ ]:
# データファイル確認
!ls -la data/splits/
!wc -l data/splits/*.ja data/splits/*.ko

In [ ]:
# トークナイザー確認
!ls -la data/tokenized/

## 3. 教師データ生成（Knowledge Distillation用）

NLLB-200を使って高品質な日本語翻訳を生成

In [ ]:
!git pull

In [ ]:
rm data/teacher/train.ja

In [ ]:
!ls data/teacher/train.ja

In [ ]:
# 韓日翻訳用教師データ生成
!python3 training/generate_teacher_data.py \
  --src_lang ko \
  --tgt_lang ja \
  --src_file data/splits/train.ko \
  --output_file data/teacher/train_ko_ja.ja \
  --batch_size 40 \
  --num_beams 3

In [ ]:
def clean_and_sync():
    ja_in = "data/raw/OpenSubtitles.ko-ja.ja"
    ko_in = "data/teacher/train.ja"
    
    ja_out = "data/clean/train.ko"
    ko_out = "data/clean/train.ja"
    
    os.makedirs("data/clean", exist_ok=True)

    with open(ja_in, 'r', encoding='utf-8') as f_ja, \
         open(ko_in, 'r', encoding='utf-8') as f_ko, \
         open(ja_out, 'w', encoding='utf-8') as o_ja, \
         open(ko_out, 'w', encoding='utf-8') as o_ko:
        
        for ja_line, ko_line in zip(f_ja, f_ko):
            ja_line = ja_line.strip()
            ko_line = ko_line.strip()
            
            # 失敗行、空行、またはエラー文字列が含まれる場合はスキップ
            if "FAILED_TRANSLATION_CLEANED" in ko_line or not ko_line:
                continue
            
            o_ja.write(ja_line + "\n")
            o_ko.write(ko_line + "\n")

    print(f"✨ クリーニング完了！")
    print(f"📁 保存先: data/clean/train.ko, train.ja")

if __name__ == "__main__":
    clean_and_sync()

In [ ]:
# !rm data/teacher/train.ja
# !wc -l data/teacher/train.ja

In [ ]:
!wc -l data/clean/train.ja
!wc -l data/clean/train.ko

In [ ]:
import random

def verify_parallel_data(ja_path, ko_path, num_samples=10):
    with open(ja_path, 'r', encoding='utf-8') as f_ja:
        ja_lines = [line.strip() for line in f_ja]
    with open(ko_path, 'r', encoding='utf-8') as f_ko:
        ko_lines = [line.strip() for line in f_ko]

    total = len(ja_lines)
    print("="*50)
    print(f"📊 データセット統計")
    print(f"  総ペア数: {total:,} 行")
    print("="*50)

    # ランダムにサンプルを抽出
    indices = random.sample(range(total), num_samples)
    
    print(f"\n🔍 ランダムサンプルチェック ({num_samples}件)")
    print("-" * 50)
    for idx in sorted(indices):
        ja = ja_lines[idx]
        ko = ko_lines[idx]
        
        # 簡易的な品質指標
        # 日本語と韓国語の長さの比率（極端に違う場合は注意）
        ratio = len(ko) / len(ja) if len(ja) > 0 else 0
        status = "✅" if 0.3 < ratio < 3.0 else "⚠️"

        print(f"[{idx+1}行目] {status}")
        print(f"  日: {ja}")
        print(f"  韓: {ko}")
        print(f"  (長さ比: {ratio:.2f})")
        print("-" * 30)

    # 最も長い文のチェック（バグ発見用）
    print("\n超長文チェック (異常な繰り返しがないか)")
    max_idx = max(range(total), key=lambda i: len(ja_lines[i]))
    print(f"[{max_idx+1}行目]")
    print(f"  日: {ja_lines[max_idx][:100]}...")
    print(f"  韓: {ko_lines[max_idx][:100]}...")

if __name__ == "__main__":
    verify_parallel_data("data/clean/train.ko", "data/clean/train.ja")

In [ ]:
# ノートブックのセルで実行
import shutil

# NLLBモデルのキャッシュディレクトリ（RunPodの標準的なパス）
cache_dir = "/workspace/huggingface_cache/models--facebook--nllb-200-3.3b"

if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"✅ {cache_dir} を削除しました。空き容量が大幅に増えたはずです。")
else:
    print("❌ 指定のパスにキャッシュが見つかりませんでした。")

In [ ]:
!df -h

In [ ]:
ls -la /workspace/huggingface_cache/

## 4. MarianMT学習

In [ ]:
!pwd

In [ ]:
!ls /workspace/grasp-models/grasp-models/

In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# データの読み込み
with open('data/clean/train.ko', 'r', encoding='utf-8') as f:
    ja_data = f.readlines()
with open('data/clean/train.ja', 'r', encoding='utf-8') as f:
    ko_data = f.readlines()

# 行数が一致しているか最終確認
if len(ja_data) == len(ko_data):
    print(f"✅ 行数一致確認: {len(ja_data)} 行")
    
    # データを分割 (学習用 98%, 検証用 1%, テスト用 1%)
    # 100万行あるので、各1%（約1万行）あれば評価には十分です
    train_ja, temp_ja, train_ko, temp_ko = train_test_split(ja_data, ko_data, test_size=0.02, random_state=42)
    val_ja, test_ja, val_ko, test_ko = train_test_split(temp_ja, temp_ko, test_size=0.5, random_state=42)

    # 保存用ディレクトリ作成
    os.makedirs('data/final', exist_ok=True)

    # 各ファイルを保存
    def save_list(path, data):
        with open(path, 'w', encoding='utf-8') as f:
            f.writelines(data)

    save_list('data/final/train.ko', train_ja)
    save_list('data/final/train.ja', train_ko)
    save_list('data/final/val.ko', val_ja)
    save_list('data/final/val.ja', val_ko)
    save_list('data/final/test.ko', test_ja)
    save_list('data/final/test.ja', test_ko)
    
    print("✅ データの分割と保存が完了しました！")
else:
    print(f"❌ 行数が一致しません: JA={len(ja_data)}, KO={len(ko_data)}")

In [ ]:
def check_alignment(ja_path, ko_path, num_samples=5):
    with open(ja_path, 'r', encoding='utf-8') as f_ja, \
         open(ko_path, 'r', encoding='utf-8') as f_ko:
        # メモリ節約のためreadlinesではなく、最小行数まで読み込み
        ja_lines = f_ja.readlines()
        ko_lines = f_ko.readlines()

    total_ja = len(ja_lines)
    total_ko = len(ko_lines)
    
    print(f"--- 統計 ---")
    print(f"日本語行数: {total_ja}")
    print(f"韓国語行数: {total_ko}")
    print(f"差分: {abs(total_ja - total_ko)}\n")

    def print_samples(indices, label):
        print(f"--- {label} のサンプル ---")
        for i in indices:
            if i < len(ja_lines) and i < len(ko_lines):
                print(f"[{i+1}行目]")
                print(f"日: {ja_lines[i].strip()}")
                print(f"韓: {ko_lines[i].strip()}")
                print("-" * 30)

    # 1. 先頭 (1〜5行目)
    print_samples(range(0, num_samples), "先頭")
    
    # 2. 中間 (50万行目あたり)
    middle_idx = 500000
    print_samples(range(middle_idx, middle_idx + num_samples), "中間")
    
    # 3. 末尾（韓国語データの最後の方）
    end_idx = total_ko - num_samples
    print_samples(range(end_idx, end_idx + num_samples), "末尾（同期可能範囲）")

# 正しいパスで実行
check_alignment('data/clean/train.ko', 'data/clean/train.ja')

In [ ]:
# テストデータ分割

# 読み込み
with open('data/clean/train.ko', 'r') as f:
    ja_lines = f.readlines()
with open('data/clean/train.ja', 'r') as f:
    ko_lines = f.readlines()

total = len(ja_lines)
train_end = total * 95 // 100
val_end = total * 975 // 1000

print(f"Total: {total}")
print(f"Train: 0 - {train_end} ({train_end} lines)")
print(f"Val: {train_end} - {val_end} ({val_end - train_end} lines)")
print(f"Test: {val_end} - {total} ({total - val_end} lines)")

# 分割して保存
with open('data/clean/train.ko', 'w') as f:
    f.writelines(ja_lines[:train_end])
with open('data/clean/train.ja', 'w') as f:
    f.writelines(ko_lines[:train_end])

with open('data/clean/val.ko', 'w') as f:
    f.writelines(ja_lines[train_end:val_end])
with open('data/clean/val.ja', 'w') as f:
    f.writelines(ko_lines[train_end:val_end])

with open('data/clean/test.ko', 'w') as f:
    f.writelines(ja_lines[val_end:])
with open('data/clean/test.ja', 'w') as f:
    f.writelines(ko_lines[val_end:])

print("\n分割完了!")

In [ ]:
!git pull

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"✅ ログイン成功: {user_info['name']} として認証されています")
except Exception as e:
    print(f"❌ ログインできていないか、エラーが発生しました: {e}")

In [ ]:
# wandbログイン（オプション）
import wandb
wandb.login()

In [ ]:
!ls data/clean

In [ ]:
# 学習実行（教師データ使用）
!python training/train.py \
    --data-dir data/clean \
    --tokenizer data/tokenized/spm.model \
    --output-dir /workspace/models/ko-ja \
    --epochs 10 \
    --batch-size 128 \
    --learning-rate 3e-4 \
    --num-workers 12

In [ ]:
# または：OPUSデータで学習（教師データなし、比較用）
# !python training/train.py \
#     --use-opus-target \
#     --output-dir /content/drive/MyDrive/grasp-models/models/ko-ja-opus \
#     --epochs 10 \
#     --batch-size 32

## 5. モデル評価

In [ ]:
import torch
from pathlib import Path
import sys
sys.path.append('/workspace/grasp-models/grasp-models')

from transformers import MarianMTModel
from training.train import SPMTokenizer

# モデルとトークナイザーをロード
model_path = "/workspace/models/ko-ja"
model = MarianMTModel.from_pretrained(model_path)
tokenizer = SPMTokenizer(f"{model_path}/spm.model")

# GPU使用
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

def translate(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128, num_beams=4)
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# テスト翻訳
test_sentences = [
    # ドラマ定番
    "私を信じて。",
    "嘘をつかないで。",
    "もう一度チャンスをください。",
    "君のことが好きだ。",
    "行かないで！",
    
    # 感情的なシーン
    "どうしてそんなことを言うの？",
    "全部私のせいだ。",
    "君がいないと生きていけない。",
    "許してくれ。",
    "二度と会いたくない。",
    
    # アクション系
    "逃げろ！",
    "後ろに誰かいる！",
    "時間がない。急げ！",
    "俺に任せろ。",
    "絶対に諦めない。",
    
    # 日常ドラマ
    "今日は残業？",
    "先輩、飲みに行きませんか？",
    "彼氏と別れたの。",
    "結婚してください。",
    "お母さん、ごめんなさい。",
]

for ja in test_sentences:
    ko = translate(ja)
    print(f"🇯🇵 {ja}")
    print(f"🇰🇷 {ko}")
    print()

## 6. ONNX変換（Android用）

In [ ]:
# 依存関係インストール
!pip install -q optimum[onnxruntime]

In [ ]:
# ONNX変換
from optimum.onnxruntime import ORTModelForSeq2SeqLM
from pathlib import Path
import shutil

model_path = "/workspace/models/ko-ja"
output_path = "/workspace/models/ko-ja-onnx"

# 変換
print("ONNX変換中...")
ort_model = ORTModelForSeq2SeqLM.from_pretrained(
    model_path,
    export=True
)

# 保存
ort_model.save_pretrained(output_path)

# トークナイザーもコピー
shutil.copy(f"{model_path}/spm.model", f"{output_path}/spm.model")

print(f"✅ 完了: {output_path}")

# サイズ確認
!ls -lh {output_path}/
!du -sh {output_path}/

In [ ]:
# 変換後のテスト
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import sys
sys.path.append('/workspace/grasp-models/grasp-models')
from training.train import SPMTokenizer

# ONNXモデルをロード
ort_model = ORTModelForSeq2SeqLM.from_pretrained(output_path)
tokenizer = SPMTokenizer(f"{output_path}/spm.model")

def translate_onnx(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = ort_model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# テスト
test = [
    "こんにちは",
    "逃げろ！",
    "君のことが好きだ。",
]

for ja in test:
    ko = translate_onnx(ja)
    print(f"🇯🇵 {ja} → 🇰🇷 {ko}")

In [ ]:
# 量子化して軽量化
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
import shutil
from pathlib import Path

output_path = "/workspace/models/ko-ja-onnx"
quantized_path = "/workspace/models/ko-ja-onnx-int8"

Path(quantized_path).mkdir(exist_ok=True)

# 量子化設定
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False)

# 全ONNXファイルを量子化
onnx_files = [
    "encoder_model.onnx",
    "decoder_model.onnx", 
    "decoder_with_past_model.onnx"
]

for onnx_file in onnx_files:
    print(f"量子化中: {onnx_file}")
    quantizer = ORTQuantizer.from_pretrained(output_path, file_name=onnx_file)
    quantizer.quantize(save_dir=quantized_path, quantization_config=qconfig)

# 設定ファイルとトークナイザーをコピー
shutil.copy(f"{output_path}/config.json", quantized_path)
shutil.copy(f"{output_path}/generation_config.json", quantized_path)
shutil.copy(f"{output_path}/spm.model", quantized_path)

print("\n✅ 量子化完了!")

# サイズ確認
!ls -lh {quantized_path}/
!du -sh {quantized_path}/

In [ ]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import sys
sys.path.append('/workspace/grasp-models/grasp-models')
from training.train import SPMTokenizer

# 量子化モデルをロード
quantized_path = "/workspace/models/ko-ja-onnx-int8"
ort_model = ORTModelForSeq2SeqLM.from_pretrained(quantized_path)
tokenizer = SPMTokenizer(f"{quantized_path}/spm.model")

def translate_q(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    outputs = ort_model.generate(**inputs, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 量子化前後で品質比較
test = [
    # ドラマ定番
    "私を信じて。",
    "嘘をつかないで。",
    "もう一度チャンスをください。",
    "君のことが好きだ。",
    "行かないで！",
    
    # 感情的なシーン
    "どうしてそんなことを言うの？",
    "全部私のせいだ。",
    "君がいないと生きていけない。",
    "許してくれ。",
    "二度と会いたくない。",
    
    # アクション系
    "逃げろ！",
    "後ろに誰かいる！",
    "時間がない。急げ！",
    "俺に任せろ。",
    "絶対に諦めない。",
    
    # 日常ドラマ
    "今日は残業？",
    "先輩、飲みに行きませんか？",
    "彼氏と別れたの。",
    "結婚してください。",
    "お母さん、ごめんなさい。",
]

print("【量子化モデル テスト】\n")
for ja in test:
    ko = translate_q(ja)
    print(f"🇯🇵 {ja}")
    print(f"🇰🇷 {ko}")
    print()

## 7. ダウンロード

学習済みモデルは `workspace/models/` に保存されています。

必須ファイル:
- encoder_model_quantized.onnx (35MB)
- decoder_model_quantized.onnx (57MB)
- spm.model (807KB)

高速推論用（推奨）:

- decoder_with_past_model_quantized.onnx (54MB)

設定ファイル:

- config.json
- generation_config.json